In [1]:
!pip install pyresparser fitz PyMuPDF pypdf3 >/dev/null             #BASH code. Run onces per computer

In [2]:
!ln -s "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf"                 #BASH code. checks if the file exists

ln: ./NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf: File exists


In [3]:
#create a string object named header. 
header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison	L.	S.	Aire géogr.	Notes'
columns = header.split('	')
columns

['Family',
 'Genus',
 'epithet',
 'author',
 'syn.',
 'Plate',
 'description',
 'Floraison',
 'L.',
 'S.',
 'Aire géogr.',
 'Notes']

In [4]:
import fitz, pprint, re

doc = fitz.open("NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf")       #open PDF and assign variable, name it "doc"
search_term = "the"                   #assign variable
#dir(doc)                             #?
#doc.getToC()                         #create a table of contents from doc file(PyMuPDF package)
page = doc[528]                       #get the 528th item. 

In [ ]:
#object.method 

In [5]:
####FUNCTION 1
#task: go through and parse out everything to its simplest form

#create a function. Opening a PDF in a text editor is not readable. this makes it so we can.  
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics
def flags_decomposer(flags):               #define a function and name it flags_decomposer
    """Make font flags human readable."""
    l = []                           #create an empty list named l
    if flags & 2 ** 0:
        l.append("superscript")      #append "superscript" to the list you just created
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)                 #return tells it to save what is produced by the function


In [6]:
####FUNCTION 2?
class Face:                                           #creating a type object "Face"
    def __init__(self, font, size, color, flags=0):   #creates an object of type Face, (default value for flags will be 0)
        self.font = font                              #self is an argument that refers to the object. "self" is a part of how to make class in Python
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
        
    def __str__(self):                          #making a string (will be used when printing a object)
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times",10.929, 0, 4))               #making and printing a new Face object 





Font: 'Times' (serifed, proportional), size 10.929, color #000000


In [37]:
###FUNCTION 3
#write a function for pulling out font that matches how the families are written 
def isfamily(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if re.match('^[A-Z]+$', t) or str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
    


In [8]:
item = dict.fromkeys(columns)              #make new dictionary from column values (from above) **twe will be pulling from the PDF and storing the values in this Dictionary (eventually turn this into CSV)
print(item)

{'Family': None, 'Genus': None, 'epithet': None, 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}


In [9]:
p = (doc[x] for x in range(528,607))    #create a sequence of integers from 528-607
print(p)

<generator object <genexpr> at 0x111878cd0>


In [10]:
###FUNCTION 5 
#make floraison function                                         #define all function at one time (seperate script)
def floraison(doc, pagenum=0, block_start=0):
    for page in next(p):
        blocks = page.getText("dict", flags=11)["blocks"]
        for x,b in enumerate(blocks[block_start:]):              # iterate through the text blocks
            for y,l in enumerate(b["lines"]):                    # iterate through the text lines
                for z,s in enumerate(l["spans"]):                # iterate through the text spans
                    s["style"] = flags_decomposer(s["flags"])    # readable font flags
                    face = Face(s["font"], s["size"], s["color"], s["flags"])
                    if re.match("Floraison", s["text"]):
                        yield (page.number, x, y, z, s["text"])


<generator object <genexpr> at 0x10d9c8350>

In [43]:
p = (doc[x] for x in range(527,529))
p


# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics
page = next(p)         #Retrieve the next item from the iterator.

# read page text as a dictionary, suppressing extra spaces in CJK fonts
# these first three for loops essentially print out everything on one page. 
blocks = page.getText("dict", flags=11)["blocks"]
for x,b in enumerate(blocks):             # iterate through the text blocks (defining values that x and b will take over the course of the "for" loop)
    for y,l in enumerate(b["lines"]):     # iterate through the text lines (iterating over the "lines" within "b")
        for z,s in enumerate(l["spans"]): # iterate through the text spans (string of characters that are sep by white space)
                                          # "s" is a stand in variable for "spans", an unbroken set of characters that its reading from the page
            #print(s)      #creates a dictonary per each word/set of words. Assigns a variable for each of these keys:
                           #{size, flags, font, color, 'text', 'bbox'}            
            s["style"] = flags_decomposer(s["flags"])  # from function we defined above. readable font flags
            #print(s)       # aguments the above output with a new key, "style" that is more descriptive
            face = Face(s["font"], s["size"], s["color"], s["flags"])   #use the class, Face, we created above. <class '__main__.Face'>
            #print(face)        #output: Font: 'Times-Roman' (serifed, proportional), size 8.95489, color #000000
                                #ie. "face" is all of the font information as one variable defined by the key, "face"? 
###Asign family to item###            
            if isfamily(s["text"], face) and len(l["spans"]) == 1:       #looking through one "span"
                item['Family'] = s["text"]
                #print(x, len(blocks), y, z, s["text"], face)             #???
                   #output: 15 18 0 0 CARYOPHYLLACEAE Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000
                #print(item)
                   #!!!!!!output: {'Family': 'CARYOPHYLLACEAE', 'Genus': None, 'epithet': None, 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}
                break       
            if not item['Family']:
                #print(item)                 #what prints is how many items are not 'family'
                break                        #moves loop to next step/line
###Assign Genus and epithet to item###         This can be made into a function for better readablily, esp bc this is not enough to parse all genus species out correctly   
            if s['font'] == 'Times-Bold':    #if the font in s is times-bold, do the following:  
                item['Genus'], *item['epithet'] = s["text"].split(' ')         #split first return in "text." splits at the space, first goes into key, "Genus", and 2nd goes into key "epithet"
                item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]   
                currsize = s['size']                 
                print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
                
#                 nn, xx, yy, zz, tt = next(floraison(page, x))
#                item['Floraison'] = tt
#                 for q in range(x, max(x+15, len(blocks))):
#                     for ll in blocks[q]["lines"]:
#                         s = ll["spans"][0]
#                         text, *rest = s['text'].split(' ')
#                         if text == "Floraison:":
#                             face = Face(s["font"], s["size"], s["color"], s["flags"])
#                             print(face, s['text'])
#                             for ss in ll["spans"]:
#                                 #print(ss['text'], end=" ")
#                                 pass
#                 from pprint import pprint
#                 #print(x, len(blocks), y, z, pprint(item))
#                 #print([a["text"] for a in l["spans"]])
#                 #print(); print(); 
#                 break
                print(item) #prints in the order of the page layout

1 18 0 1 0 Portulacca Font: 'Times-Bold' (serifed, proportional, bold), size 11, color #000000
{'Family': 'CARYOPHYLLACEAE', 'Genus': 'Portulacca', 'epithet': ' oleracea', 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}
1 18 0 1 1  oleracea Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000
{'Family': 'CARYOPHYLLACEAE', 'Genus': '', 'epithet': 'oleracea', 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}
8 18 0 1 0 Montia fontana L. Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000
{'Family': 'CARYOPHYLLACEAE', 'Genus': 'Montia', 'epithet': 'fontana', 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}
8 18 0 1 2  chondrosperma Font: 'Times-Bold' (seri

In [ ]:
search = ((x, doc.searchPageFor(x, "Floraison", 100, False)) for x in range(516,520))

In [ ]:
next(search)

In [ ]:
p = (doc[x] for x in range(516,520))

In [41]:
page = next(p)
rects = page.searchFor("Floraison")
rects[0]

StopIteration: 

In [ ]:
page.getTextBlocks()

In [ ]:
d = iter(doc)

In [ ]:
next(d).getTextBlocks()

In [42]:
pprint(p)

TypeError: 'module' object is not callable

In [ ]:
getTextBlocks621 = doc[621].getTextBlocks()

In [ ]:
getText621 = doc[621].getText()

In [ ]:
page.getFontList()

In [9]:
#write whatever data as a spreadsheet... 
#!pip install pandas                   #preinstalled with anaconda
import pandas as pd
p = pd.DataFrame(p)
p.to_csv ('p.csv')

In [36]:
csv.writer(p)

""


In [39]:
p.save("doc.pdf")

AttributeError: 'generator' object has no attribute 'save'

In [38]:
%whos

Variable           Type         Data/Info
-----------------------------------------
Face               type         <class '__main__.Face'>
columns            list         n=12
data               dict         n=0
doc                Document     fitz.Document('NOUVELLE F<...>IBAN ET DE LA SYRIE.pdf')
document           str          document.pdf
filename           str          document.pdf
fitz               module       <module 'fitz' from '/opt<...>ckages/fitz/__init__.py'>
flags_decomposer   function     <function flags_decomposer at 0x118a1a680>
floraison          function     <function floraison at 0x118a1a560>
header             str          Family	Genus	epithet	auth<...>n	L.	S.	Aire géogr.	Notes
isfamily           function     <function isfamily at 0x118a1aef0>
item               dict         n=12
p                  generator    <generator object <genexpr> at 0x118a22d50>
page               Page         page 528 of NOUVELLE FLOR<...> LIBAN ET DE LA SYRIE.pdf
pd                 mod

In [33]:
doc.searchPageFor(528, 'Pteranthus')    #search for a string on a page

[Rect(37.68000030517578, 99.17700958251953, 113.67733764648438, 113.85101318359375),
 Rect(37.91999816894531, 164.3159942626953, 94.519287109375, 179.55099487304688)]